<a href="https://colab.research.google.com/github/bhargavpirates/MultiDomainReview_SentimentAnalysis/blob/master/MultiDomain_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment analysis on DomainSentiment Data


### import Libraries

In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import pickle

from tqdm import tqdm
import os

### Keras Libraries


In [2]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


### working this problem on googleCollab So mounting Google DRive data into Collab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp -r  "/content/drive/My Drive/domain_sentiment_data.tar/sorted_data_acl/dvd" "dvd"

## From XML Data extracting requried ReviewText Data and storing them in a .txt file

In [5]:
data_list=["dvd/","books/","electronics/","kitchen_&_housewares/"]
for idx in range(4):

    pos_file = data_list[idx] +"positive.review"
    neg_file = data_list[idx] +"negative.review"
    
    print("postive file_name ::: {}".format(pos_file))
    print("negative file_name ::: {}".format(neg_file))
    
    with open(pos_file,'r',encoding='utf-8') as f:
        pos = f.readlines()
    with open(neg_file,'r',encoding='utf-8') as f:
        neg = f.readlines()
    
    lst = []
    final_pos=[]
    final_neg=[]
    
    for i in range(len(pos)):
        if(pos[i]=="<review_text>\n"):
            lst.append(i)
        elif(pos[i]=="</review_text>\n"):
            lst.append(i)   
        if(len(lst)==2):
            a=pos[ lst[0]+1 : lst[1] ]
            stng=" ".join(a)
            lst.clear()
            final_pos.append(stng.replace("\n",""))
            
    for i in range(len(neg)):
        if(neg[i]=="<review_text>\n"):
            lst.append(i)
        elif(neg[i]=="</review_text>\n"):
            lst.append(i)
    
        if(len(lst)==2):
            a=neg[ lst[0]+1 : lst[1] ]
            stng=" ".join(a)
            lst.clear()
            final_neg.append(stng.replace("\n",""))
     
    #print(len(final_pos))
    #print(len(final_neg))
    
    train , test = [], []  
    train=final_pos[:int(len(final_pos)*0.8)] + final_neg[:int(len(final_neg)*0.8)]
    test=final_pos[int(len(final_pos)*0.8):] + final_neg[int(len(final_neg)*0.8):]
    
    train_ylabel = [1 for i in range(int(len(final_pos)*0.8))] + [0 for i in range(int(len(final_pos)*0.8))]
    
    print("total number of rows in training file :::: {}".format(len(train)))
    print("total number of rows in training file :::: {}".format(len(test)))
    print(len(train_ylabel))
    

    train_file = data_list[idx]+"trainnew.txt"
    test_file = data_list[idx]+"testnew.txt"
    
    with open(train_file,'w',encoding='utf-8') as f:
        for i in range(len(train)):
            f.write(train[i])
            f.write("\n")
            
    with open(test_file,'w',encoding='utf-8') as f:
        for i in range(len(test)):
            f.write(test[i])
            f.write("\n")
    
    print()

postive file_name ::: dvd/positive.review
negative file_name ::: dvd/negative.review
total number of rows in training file :::: 1600
total number of rows in training file :::: 400
1600

postive file_name ::: books/positive.review
negative file_name ::: books/negative.review
total number of rows in training file :::: 1600
total number of rows in training file :::: 400
1600

postive file_name ::: electronics/positive.review
negative file_name ::: electronics/negative.review
total number of rows in training file :::: 1600
total number of rows in training file :::: 400
1600

postive file_name ::: kitchen_&_housewares/positive.review
negative file_name ::: kitchen_&_housewares/negative.review
total number of rows in training file :::: 1600
total number of rows in training file :::: 400
1600



### importing nltk library

In [6]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()    

stopwords=set(stopwords.words('english'))        
no_stopwords=["against","aren't","couldn't",'didn',"didn't", "doesn't", "don't", 'few', "hadn't", "hasn't", "haven't","isn't" ,'just',"mightn't",'more','most',
 "mustn't","no","nor","not","needn't","once","out","wasn't","weren't", "won't" , 'won', "wouldn't",'why','any','only','very']
modified_stopwords=list(set(stopwords) - set(no_stopwords))


## Preprocessing Data

In [8]:
import re

#reading file
def preprocess(filename):
    lst=[]
    print(filename)
    
    for file in filename:
      with open(file ,"r") as f:
        for line in f.read().split('\n'):
          lst.append(line)
          
    #print("length of lst  :::: {} ".format(len(lst)))
    lst = [ i.replace("aren't","are not").replace("couldn't","could not").replace("wasn't","was not") for i in lst ]
    lst = [ i.replace("weren't","were not").replace("mustn't","must not").replace("won't","not").replace("wouldn't","would not") for i in lst ]
    lst = [re.sub(r'[^a-zA-Z ]',r'' , (" ".join([lemmatizer.lemmatize(i.lower()) for i in line.split()  if i not in modified_stopwords])) ) for line in lst ]
    lst = [line for line in lst if line!='']
    #lst = lst[:-1]
    
    return lst
    

x_train=preprocess(["dvd/trainnew.txt","books/trainnew.txt","electronics/trainnew.txt","kitchen_&_housewares/trainnew.txt"])
x_test=preprocess(["dvd/testnew.txt","books/testnew.txt","electronics/testnew.txt","kitchen_&_housewares/testnew.txt"])

y_train=[]
y_train= [1 if(i<800) else 0 for i in range(1600)]*4
y_test= [1 if(i<200) else 0 for i in range(400)]*4

print("length of x_train ::: {}".format(len(x_train)))
print("length of x_test ::: {}".format(len(x_test)))
print("length of y_train ::: {}".format(len(y_train)))
print("length of y_test ::: {}".format(len(y_test)))

['dvd/trainnew.txt', 'books/trainnew.txt', 'electronics/trainnew.txt', 'kitchen_&_housewares/trainnew.txt']
['dvd/testnew.txt', 'books/testnew.txt', 'electronics/testnew.txt', 'kitchen_&_housewares/testnew.txt']
length of x_train ::: 6400
length of x_test ::: 1600
length of y_train ::: 6400
length of y_test ::: 1600


### Creating Frequency of words for entire reviews and sorting them by highest Frequecy Word

In [9]:
from collections import Counter

words=[i for review in x_train for i in review.split()]
frequency_words=Counter(words)

import operator

sorted_x = sorted(frequency_words.items(), key=operator.itemgetter(1))
sorted_x.reverse()
print(len(sorted_x))

desc_words=[i[0] for i in sorted_x]

38571


## converting each Sentence into Numerical vectors

In [10]:
print(type(desc_words))
print(len(desc_words))

X_train_sentence_in_num=[]
list_text=x_train
for j in tqdm(range(len(list_text))):
  sentence_in_vec=[]
  for i in list_text[j].split():
    if(i in desc_words):
      val=(desc_words.index(i))+1
      sentence_in_vec.append(val)
    else:
      sentence_in_vec.append(0)
  #if(i in desc_words):
  X_train_sentence_in_num.append(sentence_in_vec)

  0%|          | 7/6400 [00:00<02:03, 51.92it/s]

<class 'list'>
38571


100%|██████████| 6400/6400 [01:07<00:00, 94.65it/s] 


In [11]:
X_train_sentence_in_num[1]

[34, 25, 62, 79, 451, 439, 44, 586, 73, 319, 2519, 24, 112, 1161]

In [12]:
print(type(desc_words))
print(len(desc_words))

X_test_sentence_in_num=[]
list_text=x_test
for j in tqdm(range(len(list_text))):
  sentence_in_vec=[]
  for i in list_text[j].split():
    if(i in desc_words):
      val=(desc_words.index(i))+1
      sentence_in_vec.append(val)
    else:
      sentence_in_vec.append(0)
  #if(i in desc_words):
  X_test_sentence_in_num.append(sentence_in_vec)


  1%|          | 10/1600 [00:00<00:16, 98.82it/s]

<class 'list'>
38571


100%|██████████| 1600/1600 [00:17<00:00, 92.02it/s] 


In [13]:
print(len(X_train_sentence_in_num))
print(len(X_test_sentence_in_num))

6400
1600


## padding each sentence  ie making each reviews length_of_words  same

In [14]:
# truncate and/or pad input sequences
max_review_length = 600
X_train = sequence.pad_sequences(X_train_sentence_in_num, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test_sentence_in_num, maxlen=max_review_length)

print(X_train.shape)
#print(X_train[1])

(6400, 600)


# LSTM Model

In [15]:
   # create the model
top_words=38572
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

W0721 08:56:11.143889 140275830638464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 08:56:11.197532 140275830638464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 08:56:11.205862 140275830638464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0721 08:56:11.496620 140275830638464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 08:56:11.521598 140275830638464 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           1234304   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,287,605
Trainable params: 1,287,605
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
history =  model.fit(X_train, y_train, nb_epoch=20, batch_size=128   , validation_split=0.2 )
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

W0721 08:56:12.626731 140275830638464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 5120 samples, validate on 1280 samples
Epoch 1/20
5120/5120 [==============================] - 54s 11ms/step - loss: 0.6860 - acc: 0.5727 - val_loss: 0.6755 - val_acc: 0.6586
Epoch 2/20
5120/5120 [==============================] - 54s 11ms/step - loss: 0.5685 - acc: 0.7988 - val_loss: 0.6432 - val_acc: 0.6078
Epoch 3/20
5120/5120 [==============================] - 55s 11ms/step - loss: 0.3510 - acc: 0.8678 - val_loss: 0.7309 - val_acc: 0.6789
Epoch 4/20
5120/5120 [==============================] - 55s 11ms/step - loss: 0.1750 - acc: 0.9373 - val_loss: 1.0248 - val_acc: 0.6281
Epoch 5/20
5120/5120 [==============================] - 55s 11ms/step - loss: 0.0809 - acc: 0.9762 - val_loss: 1.4371 - val_acc: 0.6164
Epoch 6/20
5120/5120 [==============================] - 54s 11ms/step - loss: 0.0321 - acc: 0.9937 - val_loss: 1.7671 - val_acc: 0.6047
Epoch 7/20
5120/5120 [==============================] - 53s 10ms/step - loss: 0.0157 - acc: 0.9971 - val_loss: 1.6600 - val_acc: 0.6211


#### LSTM Model Accuracy  :::  75.50%

## Predicating unlabeled Reviews 

In [18]:
data_list=["dvd/","electronics/","kitchen_&_housewares/"]
final_unlabeled=[]
for idx in range(3):

    unlabeled_file = data_list[idx] +"unlabeled.review"
    
    print("unlabeled_file file_name ::: {}".format(unlabeled_file))
    
    with open(unlabeled_file,'r',encoding='utf-8') as f:
        unlabeled = f.readlines()
    
    lst = []
   
    for i in range(len(unlabeled)):
        if(unlabeled[i]=="<review_text>\n"):
            lst.append(i)
        elif(unlabeled[i]=="</review_text>\n"):
            lst.append(i)   
        if(len(lst)==2):
            a=unlabeled[ lst[0]+1 : lst[1] ]
            stng=" ".join(a)
            lst.clear()
            final_unlabeled.append(stng.replace("\n",""))
            
    
    print("total number of rows in final_unlabeled file :::: {}".format(len(final_unlabeled)))


unlabeled_file file_name ::: dvd/unlabeled.review
total number of rows in final_unlabeled file :::: 34741
unlabeled_file file_name ::: electronics/unlabeled.review
total number of rows in final_unlabeled file :::: 47894
unlabeled_file file_name ::: kitchen_&_housewares/unlabeled.review
total number of rows in final_unlabeled file :::: 64679


In [0]:
#x_test=preprocess(["dvd/unlabeled.review","electronics/unlabeled.review","kitchen_&_housewares/unlabeled.review"])
final_unlabeled = [ i.replace("aren't","are not").replace("couldn't","could not").replace("wasn't","was not") for i in final_unlabeled ]
final_unlabeled = [ i.replace("weren't","were not").replace("mustn't","must not").replace("won't","not").replace("wouldn't","would not") for i in final_unlabeled ]
final_unlabeled = [re.sub(r'[^a-zA-Z ]',r'' , (" ".join([lemmatizer.lemmatize(i.lower()) for i in line.split()  if i not in modified_stopwords])) ) for line in final_unlabeled ]
final_unlabeled = [line for line in final_unlabeled if line!='']

In [0]:
unlabeled_tfidf     =  tf_idf_vect.transform(final_unlabeled)
unlabeled_tfidf     =  scaler.transform(unlabeled_tfidf)
unlabel_predicted   =  clf1.predict(unlabeled_tfidf)

In [20]:
unlabeled_sentence_in_num=[]
list_text=x_test
for j in tqdm(range(len(final_unlabeled))):
  sentence_in_vec=[]
  for i in final_unlabeled[j].split():
    if(i in desc_words):
      val=(desc_words.index(i))+1
      sentence_in_vec.append(val)
    else:
      sentence_in_vec.append(0)
  #if(i in desc_words):
  unlabeled_sentence_in_num.append(sentence_in_vec)

100%|██████████| 64679/64679 [14:15<00:00, 75.59it/s] 


In [0]:
max_review_length = 600
unlabeled = sequence.pad_sequences(unlabeled_sentence_in_num, maxlen=max_review_length)

In [0]:
prediction = model.predict(np.array(unlabeled))

In [31]:
prediction

array([[0.9475956 ],
       [0.69510585],
       [0.05466333],
       ...,
       [0.9856909 ],
       [0.99397814],
       [0.92629343]], dtype=float32)

In [0]:
result = np.where(prediction < 0.5, "Negative", "Postive" )
print(result)